In [ ]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

--2023-04-10 17:07:55--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.160.249.45, 18.160.249.78, 18.160.249.12, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.160.249.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   165MB/s    in 24s     

2023-04-10 17:08:19 (104 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [ ]:
!pip install torch torchvision
!pip install opencv-python matplotlib
!pip install 'git+https://github.com/facebookresearch/segment-anything.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-y6q_izr0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-y6q_izr0
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 06bd20da89b5ec4b0f0256b3197596d4cd5c2f8a
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36584 sha256=b75fafbba33e653d9fda5a6f8c96104dfc739fdad079db2b7c27ec299e97bb14
  Stored in directory: /tmp/pip-ephem-wheel-cache-qhkoyk9s/wheels/d5/11/03/7aca746a2c0e09f279b10436ced7175926bc38f650b736a648
Successfull

In [ ]:
import torch
import torchvision
import cv2
import numpy as np
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# Initialize YOLOv5
model = torch.hub.load("ultralytics/yolov5", "yolov5s", pretrained=True).to("cuda")

/usr/local/lib/python3.9/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: YOLOv5 requirement "gitpython>=3.1.30" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 5.

RuntimeError: ignored

In [ ]:
# Initialize Segment-Anything
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
mask_generator_ = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.9,
    stability_score_thresh=0.96,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,
)

In [ ]:
# User input class to blur
class_to_blur = "person"

# Load an image
input_image = "trifecta.jpg"
image = cv2.imread(input_image)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Make a copy of the original image
original_image = image_rgb.copy()

# Perform YOLOv5 object detection
detections = model(image_rgb)
detections_list = detections.xyxy[0].tolist()

# Process detections
for det in detections_list:
    x1, y1, x2, y2, conf, cls = det
    class_name = model.names[int(cls)]

    if class_name == class_to_blur:
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cropped_frame = image_rgb[y1:y2, x1:x2].copy()
        masks = mask_generator_.generate(cropped_frame)
        for mask in masks:
            segmentation = mask["segmentation"]
            blurred_region = cv2.GaussianBlur(cropped_frame, (99, 99), 30)
            cropped_frame = np.where(segmentation[..., None], blurred_region, cropped_frame)
        image_rgb[y1:y2, x1:x2] = cropped_frame


# Save the blurred image
output_image_path = "output_image.jpg"
output_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
cv2.imwrite(output_image_path, output_image)


True

Test:

In [ ]:
# User input class to blur
class_to_blur = "person"

# Load an image
input_image = "trifecta.jpg"
image = cv2.imread(input_image)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Make a copy of the original image
original_image = image_rgb.copy()

# Perform YOLOv5 object detection
detections = model(image_rgb)
detections_list = detections.xyxy[0].tolist()

# Process detections
for det in detections_list:
    x1, y1, x2, y2, conf, cls = det
    class_name = model.names[int(cls)]

    if class_name == class_to_blur:
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cropped_frame = image_rgb[y1:y2, x1:x2].copy()
        masks = mask_generator_.generate(cropped_frame)
        
        # Find the largest mask
        largest_mask = None
        largest_mask_area = 0
        for mask in masks:
            segmentation = mask["segmentation"]
            mask_area = np.sum(segmentation)
            if mask_area > largest_mask_area:
                largest_mask_area = mask_area
                largest_mask = mask
        
        if largest_mask is not None:
            # Apply blur only on the largest mask
            segmentation = largest_mask["segmentation"]
            blurred_region = cv2.GaussianBlur(cropped_frame, (99, 99), 30)
            cropped_frame = np.where(segmentation[..., None], blurred_region, cropped_frame)
            image_rgb[y1:y2, x1:x2] = cropped_frame

# Save the blurred image
output_image_path = "output_image.jpg"
output_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
cv2.imwrite(output_image_path, output_image)